## Task description


Find the prices of a EU call, with S0 = K = 50, r = 0.1 (annual), σ = 0.4,
and maturity in five months using Monte Carlo methods, but use
antithetic variates, stratified sampling (use two strata, no need to use the
optimal weight), and control variates (use the underlying price as control)
as variance reduction techniques. Report the confidence intervals.



Use Quasi-Monte Carlo method to price the same EU call above (both
Halton and Weyl sequence). Report the confidence intervals.


In [12]:
# Import packages
import numpy as np
from scipy.stats import truncnorm, norm  # Truncated and normal continuous random variable
from scipy.stats import qmc

## MC Method

In [10]:
# Defining the pricing for EU Call by using Monte Carlo

np.random.seed(42)
def european_call_monte_carlo():
    """
    Monte Carlo Simulation for European Call Option with the following variance reduction techniques:
    - Antithetic Variates
    - Stratified Sampling (two strata)
    - Controll Variates (using the underlying asset price)
    """
    # ====================================================
    # Parameter initialization
    # ====================================================
    S0 = 50
    K = 50
    r = 0.1
    sigma = 0.4
    T = 5/12
    N = 16000

    # Ensure N is a multiple of 4 for stratification and antithetic variates
    if N % 4 !=0:
        raise ValueError("Number of simulations N must be a multiple of 4")

    M = N // 4

    # ====================================================
    # Stratified Sampling Setup
    # ====================================================
    # Define truncation points for two strata, e.g we divide the standard normal distribution into two strata
    # Stratum 1: Z <= 0
    a1, b1 = -10 , 0

    # Stratum 2: Z > 0
    a2, b2 = 0, 10

    # Generate M samples for Stratum 1
    Z1 = truncnorm(a=(a1 - 0)/1, b=(b1 - 0)/1, loc=0, scale=1).rvs(M)
    Z1_antithetic = -Z1
    Z_stratum1 = np.concatenate([Z1, Z1_antithetic])

    # Generate M samples for Stratum 2
    Z2 = truncnorm(a=(a2 - 0)/1, b=(b2 - 0)/1, loc=0, scale=1).rvs(M)
    Z2_antithetic = -Z2
    Z_stratum2 = np.concatenate([Z2, Z2_antithetic])

    # Combine both strata
    Z = np.concatenate([Z_stratum1, Z_stratum2])

    # ====================================================
    # Asset Price Simulation
    # ====================================================
    S_T = S0 * np.exp((r - 0.5*sigma**2) * T + sigma * np.sqrt(T)*Z)

    # ====================================================
    # Payoff Calculation
    # ====================================================
    payoff = np.maximum(S_T - K, 0)

    # ====================================================
    # Control Variates Technique
    # ====================================================
    E_S_T = S0 * np.exp(r * T)

    # Calculate sample covariance between payoff and S_T
    cov_matrix = np.cov(payoff, S_T, ddof=1)    # Unbiased estimate
    cov_payoff_S = cov_matrix[0, 1]

    # Calculate sample variance of S_T
    var_S = np.var(S_T, ddof=1)

    # Optimal coefficient for control variate
    c = cov_payoff_S / var_S

    # Adjust payoff's using control variate
    adjusted_payoff = payoff + c * (E_S_T - S_T)

    # ====================================================
    # Discount and Statistics (Variance Reduction)
    # ====================================================

    # Discount average adjusted payoff to present value (variance Reduction)
    option_price = np.exp(-r * T)*np.mean(adjusted_payoff)

    # Calculate the standard error of the adjusted payoffs
    std_error= np.exp(-r * T) * np.std(adjusted_payoff, ddof=1) / np.sqrt(N)

    # Compute the 95% CI
    conf_interval = (option_price - 1.96 * std_error, option_price + 1.96 * std_error)

    # ====================================================
    # Discount and Statistics (Basic MC)
    # ====================================================
    basic_payoff = np.maximum(S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * np.random.randn(N)) - K, 0)
    basic_option_price = np.exp(-r * T) * np.mean(basic_payoff)

    # ====================================================
    # Output Results
    # ====================================================
    print(f"European Call Option Price (Variance Reduction): {option_price:.4f}")
    print(f"European Call Option Price (Basic MC): {basic_option_price:.4f}")
    print(f"95% CI: [{round(conf_interval[0], 2), round(conf_interval[1], 2)}]")



if __name__ == "__main__":
    european_call_monte_carlo()

European Call Option Price (Variance Reduction): 6.0876
European Call Option Price (Basic MC): 6.1789
95% CI: [(6.03, 6.14)]


## Quasi-Monte Carlo (both Halton and Weyl sequence)

In [19]:
# We need to build the Weyl Sequences ourselves 

def generate_weyl_sequence(N, shift=0.5):
    """
    Generates a Weyl Sequence using a deterministic approach based on the golden ratio conjugate.
    Returns: A sequence of points (N) in [0, 1]
    """

    # Golden ratio conjugate
    alpha = (np.sqrt(5) - 1) / 2

    # Generate the Weyl sequence
    sequence = (np.arange(1, N, +1)* alpha + shift) % 1
    return sequence

In [20]:
# Defining the pricing for EU Call by using QMC by either Halton or Weyl
def price_european_call_qmc(S0, K, r, sigma, T, N, sequence_type = "halton"):

# ====================================================
# Sequence Type
# ====================================================
    if sequence_type.lower() == "halton":
        # Initialize Halton sampler
        sampler = qmc.Halton(d=1, scramble=True, seed=42)
        sample = sampler.random(N)
        U = sample.flatten()
    else:
        U = generate_weyl_sequence(N)

# ===============================================================
# Transforming to standard dnormal using inverse CDF
# ===============================================================
    Z = norm.ppf(U)

# Handle infinities resultin from norm.ppf(0) or norm.ppf(1)
    Z = np.where(Z == -np.inf, -10, Z)
    Z = np.where(Z == np.inf, 10, Z)

# ===============================================================
# Asset price at maturity using GBM, payoffs price and statistics
# ===============================================================
    S_T = S0 * np.exp((r - 0.5 * sigma**2)* T + sigma * np.sqrt(T) * Z)

# Payoff for EU call
    payoff = np.maximum(S_T - K, 0)

# Discount payoffs to present value
    discounted_payoff = np.exp(-r * T) * payoff

# Estimate option price as the mean of discounted payoffs
    option_price = np.mean(discounted_payoff)

# Standard Error using Sample
    std_error = np.std(discounted_payoff, ddof=1) / np.sqrt(N)

# Compute 95% CI
    conf_interval = (option_price- 1.96*std_error, option_price + 1.96*std_error)

    return option_price, conf_interval

def main():
    # Option Parameters
    S0 = 50            # Initial stock price
    K = 50             # Strike price
    r = 0.1            # Risk-free interest rate (annual)
    sigma = 0.4        # Volatility (annual)
    T = 5/12           # Time to maturity in years (5 months)
    N = 16000          # Number of simulations
    
    # Price using Halton sequence
    halton_price, halton_conf = price_european_call_qmc(
        S0, K, r, sigma, T, N, sequence_type='halton'
    )
    
    # Price using Weyl sequence
    weyl_price, weyl_conf = price_european_call_qmc(
        S0, K, r, sigma, T, N, sequence_type='weyl'
    )
     # Output Results
    print("European Call Option Pricing using Quasi-Monte Carlo Methods")
    print("-----------------------------------------------------------")
    print(f"Number of Simulations: {N}\n")
    
    print("1. Halton Sequence:")
    print(f"   Option Price Estimate: {halton_price:.4f}")
    print(f"   95% Confidence Interval: [{halton_conf[0]:.4f}, {halton_conf[1]:.4f}]\n")
    
    print("2. Weyl Sequence:")
    print(f"   Option Price Estimate: {weyl_price:.4f}")
    print(f"   95% Confidence Interval: [{weyl_conf[0]:.4f}, {weyl_conf[1]:.4f}]\n")


if __name__ == "__main__":
    main()

European Call Option Pricing using Quasi-Monte Carlo Methods
-----------------------------------------------------------
Number of Simulations: 16000

1. Halton Sequence:
   Option Price Estimate: 6.1165
   95% Confidence Interval: [5.9701, 6.2629]

2. Weyl Sequence:
   Option Price Estimate: 6.1172
   95% Confidence Interval: [5.9708, 6.2636]

